# Open ai trading bot using image and data with order management
Process Overview:
Data Fetching: Retrieve forex data using the OANDA API.
Indicator Calculation: Calculate indicators like RSI and plot them.
Analysis: Send data to OpenAI's GPT-4o for analysis and receive trading signals.
Order Placement: Place orders based on the analysis results.
Order Management:
Monitoring: Regularly check open positions and update stop-loss levels dynamically.
Risk-Free Trades: Adjust stop-loss to entry price once in profit to ensure risk-free trades.
Dynamic Adjustments: Update SL and TP based on new predictions.
Multiple Orders: Manage multiple open positions efficiently.
Comprehensive Monitoring: Continuously monitor market conditions and adjust positions as needed.
This combined script should cov

In [1]:
import os
import json
import time
import logging
import base64
import requests
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta, timezone
from PIL import Image
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.endpoints.orders import OrderCreate, OrderReplace
from oandapyV20.endpoints.positions import PositionDetails
from oandapyV20.endpoints.pricing import PricingInfo
from oandapyV20.exceptions import V20Error
import openai
import tiktoken
from oandapyV20.endpoints.pricing import PricingInfo

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

# Parameters for real-time data fetching and processing
granularity = 'M15'  # 15 minutes granularity
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '15 minutes'

# Fetch forex data
def fetch_forex_data(from_date, to_date, granularity, instrument):
    logging.info(f"Fetching forex data from {from_date} to {to_date} with granularity {granularity} for instrument {instrument}")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = api.request(request)
            if response:
                for candle in response.get('candles'):
                    rec = {
                        'time': candle.get('time')[0:19],
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    return pd.DataFrame(data)

# Calculate RSI
def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Plot candlestick chart
def plot_candlestick_chart(df, filename):
    df.index = pd.to_datetime(df['time'])
    df.index.name = 'Date'
    
    # Calculate support and resistance
    support = df['low'].rolling(window=20).min()
    resistance = df['high'].rolling(window=20).max()

    # Calculate RSI
    df['RSI'] = calculate_rsi(df['close'])
    
    if df['RSI'].dropna().empty:
        logging.error("RSI calculation resulted in empty data. Skipping plot.")
        return

    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--', y_on_right=False)

    addplots = [
        mpf.make_addplot(support, color='blue', linestyle='dashed'),
        mpf.make_addplot(resistance, color='orange', linestyle='dashed'),
        mpf.make_addplot(df['RSI'], panel=1, color='purple', secondary_y=False),
    ]
    
    kwargs = dict(
        type='candle', 
        style=s, 
        addplot=addplots, 
        volume=True, 
        figscale=2.5,  # Further increase the size of the candles
        figratio=(10, 8),  # Adjust figure ratio
        title=pair, 
        ylabel='Price', 
        ylabel_lower='Volume', 
        panel_ratios=(2, 1),  # Allocate more space for the candlestick chart compared to RSI
        tight_layout=True,  # Remove whitespace around the chart
        fontscale=1.2,  # Increase font size for better readability
    )
    
    mpf.plot(df, **kwargs, savefig=dict(fname=filename, dpi=100, pad_inches=0.1))

# Compress image
def compress_image(input_path, output_path, quality=85):
    with Image.open(input_path) as img:
        img = img.convert('RGB')  # Convert to RGB
        img = img.resize((510, 510), Image.LANCZOS)  # Resize to 510x510
        img.save(output_path, 'JPEG', quality=quality)  # Save with reduced quality

# Encode image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Save prompt to txt
def save_prompt_to_txt(prompt_content, filename):
    with open(filename, 'w') as file:
        json.dump(prompt_content, file, indent=4)

# Analyze data with GPT-4o
def analyze_data_with_gpt4o(price_data, image_base64):
    logging.info("Sending data to OpenAI API for analysis")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    prompt_content = {
        "content": (
            f"Here is the EUR/USD close price data from {price_data['start_date']} to {price_data['end_date']} at a 15-minute interval. Analyze this data and the provided chart to calculate RSI, MACD, Bollinger Bands, and Fibonacci Retracement. Use these indicators, along with the chart, to perform a comprehensive analysis and identify potential trading opportunities. Assign a probability score from 0 to 100 and profit/loss rate for all patterns. Follow these steps:\n"
            "1. Analyze the chart visually and identify possible patterns without considering additional data.\n"
            "2. Using both the chart and calculated indicators, identify possible patterns.\n"
            "3. Analyze the data from the indicators independently and list all possible predictions.\n"
            "4. Combine the results from the three types of analysis (chart-only, chart with data, data-only) to form potential orders. Select the best order based on:\n"
            "   - Majority of detected patterns and predictions.\n"
            "   - Higher benefit-to-loss ratio.\n"
            "   - Predicted profit (minimum 20 to 50 pips).\n"
            "Provide the analysis in JSON format:\n"
            "{\n"
            "   \"orders\": [\n"
            "        {\n"
            "            \"timeframe\": \"15 minutes\",\n"
            "            \"pattern_name\": \"Pattern Name\",\n"
            "            \"confidence_percentage\": xx,\n"
            "            \"action\": \"Buy/Sell\",\n"
            "            \"entry_price\": x.xxxx,\n"
            "            \"take_profit\": x.xxxx,\n"
            "            \"stop_loss\": x.xxxx,\n"
            "            \"profit_loss_ratio\": x.x,\n"
            "            \"deadline_date\": \"yyyy-mm-ddThh:mm:ssZ\"\n"
            "        }\n"
            "    ],\n"
            "    \"best_pattern\": {\n"
            "        \"pattern_name\": \"Best Pattern Name\",\n"
            "        \"confidence_percentage\": xx,\n"
            "        \"profit_loss_rate\": x.x,\n"
            "    }\n"
            "}\n"
        ),
        "price_data": {
            "close_prices": price_data['close_prices']
        },
        "image": f"data:image/png;base64,{image_base64}"
    }

    # Calculate tokens before adding image
    enc = tiktoken.encoding_for_model("gpt-4o")
    num_tokens = len(enc.encode(json.dumps(prompt_content)))
    data_cost = num_tokens * 0.000005
    image_cost = 0.001275  # Fixed image cost based on 512x512 px
    
    logging.info(f"Data tokens before adding image: {num_tokens}, Estimated Data Cost: ${data_cost:.6f}")

    save_prompt_to_txt(prompt_content, 'final_prompt.txt')
    
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": json.dumps(prompt_content)
            }
        ],
        "max_tokens": 3000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    try:
        response_data = response.json()
        return response_data, data_cost, image_cost
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response from OpenAI API")
        return None, data_cost, image_cost

# Extract and place order
def extract_and_place_order(response_data):
    if not response_data or "choices" not in response_data:
        logging.error("Invalid response data")
        return None

    content = response_data["choices"][0]["message"]["content"]
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_content = content[start_index:end_index]
    
    try:
        analysis = json.loads(json_content)
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse JSON content: {e}")
        return None
    
    orders = analysis.get("orders", [])
    best_pattern = analysis.get("best_pattern", {})
    
    for order in orders:
        if order.get("profit_loss_ratio", 0) > 2:
            logging.info(f"Order Details - Action: {order['action']}, Entry Price: {order['entry_price']}, Take Profit: {order['take_profit']}, Stop Loss: {order['stop_loss']}")
            
            order_details = {
                'action': order['action'],
                'entry_price': order['entry_price'],
                'take_profit': order['take_profit'],
                'stop_loss': order['stop_loss'],
                'deadline_date': order['deadline_date']
            }
            response = place_order(order_details)
            
            if 'orderCancelTransaction' in response:
                logging.info(f"Order {response['orderCancelTransaction']['orderID']} was canceled: {response['orderCancelTransaction']['reason']}")
            return order_details

    logging.info(f"Best Pattern Details - Action: {best_pattern.get('action', 'N/A')}, Entry Price: {best_pattern.get('entry_price', 'N/A')}, Take Profit: {best_pattern.get('take_profit', 'N/A')}, Stop Loss: {best_pattern.get('stop_loss', 'N/A')}")
    return best_pattern

# Place order
def place_order(order_details):
    instrument = "EUR_USD"
    
    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=-10000 if order_details['action'].upper() == 'SELL' else 10000,
        takeProfitOnFill=TakeProfitDetails(price=order_details['take_profit']).data,
        stopLossOnFill=StopLossDetails(price=order_details['stop_loss']).data
    )
    
    r = OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        response = api.request(r)
        logging.info(f"Order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing order: {e}")
        return {"error": str(e)}

# Save data to CSV
def save_data_to_csv(data, filename):
    data.to_csv(filename, index=False)

# Check if weekend
def is_weekend(date):
    return date.weekday() > 4  # Saturday and Sunday are 5 and 6

# Get last weekday
def get_last_weekday(date):
    while date.weekday() > 4:  # If it's Saturday (5) or Sunday (6)
        date -= timedelta(days=1)
    return date

# Order Management Functions
def get_open_positions(api, account_id, pair):
    # Fetch open positions from the account
    response = api.request(PositionDetails(accountID=account_id, instrument=pair))
    positions = response.get('position', [])
    if isinstance(positions, dict):  # Sometimes it's a single position, convert to list
        positions = [positions]
    return positions




def update_stop_loss(api, account_id, order_id, new_stop_loss):
    # Update the stop loss level for an open order
    order_details = {
        "order": {
            "stopLossOnFill": {
                "price": str(new_stop_loss),
                "timeInForce": "GTC"
            }
        }
    }
    request = OrderReplace(accountID=account_id, orderID=order_id, data=order_details)
    try:
        api.request(request)
        logging.info(f"Stop loss updated for order {order_id} to {new_stop_loss}")
    except V20Error as e:
        logging.error(f"Failed to update stop loss for order {order_id}: {e}")

from oandapyV20.endpoints.pricing import PricingInfo

def get_current_price(api, pair):
    # Fetch the current market price
    params = {"instruments": pair}
    response = api.request(PricingInfo(accountID=account_id, params=params))
    return float(response['prices'][0]['closeoutBid'])

def monitor_and_manage_orders(api, account_id, pair, trailing_stop_pips=10):
    open_positions = get_open_positions(api, account_id, pair)
    if not open_positions:
        logging.info(f"No open positions for {pair}")
        return

    for position in open_positions:
        logging.info(f"Inspecting position: {position}")

        if 'long' in position and position['long']['units'] != '0':
            units = position['long']['units']
            entry_price = float(position['long']['averagePrice'])
            stop_loss = float(position['long']['stopLossOrder']['price']) if 'stopLossOrder' in position['long'] else None
            trade_ids = position['long']['tradeIDs'] if 'tradeIDs' in position['long'] else None
        elif 'short' in position and position['short']['units'] != '0':
            units = position['short']['units']
            entry_price = float(position['short']['averagePrice'])
            stop_loss = float(position['short']['stopLossOrder']['price']) if 'stopLossOrder' in position['short'] else None
            trade_ids = position['short']['tradeIDs'] if 'tradeIDs' in position['short'] else None
        else:
            continue

        if not trade_ids:
            logging.warning(f"Trade IDs not found for position: {position}")
            continue
        
        trade_id = trade_ids[0]  # Use the first trade ID in the list

        current_price = get_current_price(api, pair)
        logging.info(f"Monitoring position: Entry Price: {entry_price}, Current Price: {current_price}, Units: {units}")

        # Calculate new stop loss level for trailing stop
        if float(units) > 0:  # Long position
            new_stop_loss = current_price - trailing_stop_pips * 0.0001
            if stop_loss is None or new_stop_loss > stop_loss:
                logging.info(f"Updating stop loss for long position to {new_stop_loss}")
                update_stop_loss(api, account_id, trade_id, new_stop_loss)
        else:  # Short position
            new_stop_loss = current_price + trailing_stop_pips * 0.0001
            if stop_loss is None or new_stop_loss < stop_loss:
                logging.info(f"Updating stop loss for short position to {new_stop_loss}")
                update_stop_loss(api, account_id, trade_id, new_stop_loss)

# Main execution loop
while True:
    now = datetime.now(timezone.utc)
    
    if is_weekend(now):
        logging.info("It's the weekend. Using last available weekday data.")
        now = get_last_weekday(now)
    
    start_time = (now - timedelta(days=3)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = now.strftime('%Y-%m-%dT%H:%M:%SZ')

    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    save_data_to_csv(prices, 'forex_data.csv')

    # Collect all the raw price data
    price_data = {
        "start_date": start_time,
        "end_date": end_time,
        "close_prices": prices['close'].to_list()
    }

    # Plot and encode the image
    filename = "normal_chart.png"
    compressed_filename = "compressed_chart.jpg"
    plot_candlestick_chart(prices, filename)  # Use color image
    compress_image(filename, compressed_filename, quality=85)  # Compress without grayscale
    image_base64 = encode_image(compressed_filename)

    # Analyze data and place orders based on predictions
    analysis_result, data_cost, image_cost = analyze_data_with_gpt4o(price_data, image_base64)
    logging.info(f"OpenAI API Analysis Result: {json.dumps(analysis_result, indent=4)}")

    if analysis_result:
        response_tokens = analysis_result["usage"]["completion_tokens"]
        response_cost = response_tokens * 0.000015  # Adjust based on actual token cost
        logging.info(f"Response received: {response_tokens} tokens, Cost: ${response_cost:.6f}")

        prompt_tokens = analysis_result["usage"]["prompt_tokens"]
        prompt_cost = prompt_tokens * 0.00005  # Adjust based on actual token cost
        logging.info(f"Prompt sent: {prompt_tokens} tokens, Cost: ${prompt_cost:.6f}")

        order_details = extract_and_place_order(analysis_result)
        if order_details:
            logging.info("Order placed based on analysis.")

    # Order management
    monitor_and_manage_orders(api, account_id, pair)

    total_cost = data_cost + response_cost + image_cost
    logging.info(f"Total cost for this run: Data: ${data_cost:.6f}, Image: ${image_cost:.6f}, Response: ${response_cost:.6f}, Total: ${total_cost:.6f}")

    logging.info("Waiting for 15 minutes before next run...")
    time.sleep(900)


INFO:root:Fetching forex data from 2024-06-07T00:38:42Z to 2024-06-10T00:38:42Z with granularity M15 for instrument EUR_USD
INFO:root:Sending data to OpenAI API for analysis
INFO:root:Data tokens before adding image: 32210, Estimated Data Cost: $0.161050
INFO:root:OpenAI API Analysis Result: {
    "id": "chatcmpl-9YNLYglzlSv1c86SitTwhKCfOGgjs",
    "object": "chat.completion",
    "created": 1717979924,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "{\n  \"orders\": [\n    {\n      \"timeframe\": \"15 minutes\",\n      \"pattern_name\": \"Bearish Divergence\",\n      \"confidence_percentage\": 85,\n      \"action\": \"Sell\",\n      \"entry_price\": 1.08332,\n      \"take_profit\": 1.08052,\n      \"stop_loss\": 1.08500,\n      \"profit_loss_ratio\": 1.5,\n      \"deadline_date\": \"2024-06-10T00:38:42Z\"\n    },\n    {\n      \"timeframe\": \"15 minutes\",\n 